<a href="https://colab.research.google.com/github/thwlruss10/VDA_pipeline/blob/main/BERT_DEIT_inference_20APR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers datasets nltk scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [4]:
import torch
import torch.nn as nn
from transformers import AutoModel
from typing import Optional

'''
The MultimodalVQAModel takes a question and an image as input.
It uses pre-trained models to extract features from both, combines these features,
and then predicts the answer to the question based on the combined information.
This architecture is typical for VQA tasks, where the goal is to understand both
visual and textual information to provide accurate answers.

forward() Defines how the model processes data during training and inference.
  input_ids: Tokenized representation of the question.
  pixel_values: Processed image data.
  attention_mask: Indicates which tokens in the question are relevant.
  token_type_ids: Distinguishes between different segments of text (if applicable).
  labels: The correct answer (used during training).
'''

class MultimodalVQAModel(nn.Module):  #Defining Mu-Mo-VQA model is per PyTorch base class nn.Module
    def __init__(
        self,
        pretrained_text_name: str,
        pretrained_image_name: str,
        num_labels: int,
        intermediate_dim: int = 512,  # increase to reduce overfitting can modify to 1024
        dropout: float = 0.5          # prevents overfitting, can decrease to 0.1.
    ):
        super(MultimodalVQAModel, self).__init__()

        self.num_labels = num_labels  #length of answer space
        self.text_encoder = AutoModel.from_pretrained(pretrained_text_name) # attention mechanism lives in pre-trained transformers
        self.image_encoder = AutoModel.from_pretrained(pretrained_image_name)

        # Fallback if pooler_output doesn't exist (e.g., in ViT)
        self.use_pooler_text = hasattr(self.text_encoder.config, "pooler_fc_size")
        self.use_pooler_image = hasattr(self.image_encoder.config, "pooler_fc_size")

        # Fusion + Classification layers
        # linear transformation, activation function (ReLU), dropout regularization
        # Integrates the combined features comprehensive representation
        self.fusion = nn.Sequential(
            nn.Linear(
                self.text_encoder.config.hidden_size + self.image_encoder.config.hidden_size,
                intermediate_dim
            ),
            # nn.BatchNorm1d(intermediate_dim),  # Add Batch Normalization
            nn.ReLU(),            # alt nn.LeakyReLU, nn.GELU, or nn.ELU
            nn.Dropout(dropout),
        )

        self.classifier = nn.Linear(intermediate_dim, self.num_labels)
        self.criterion = nn.CrossEntropyLoss()
    # Feed forward fusion
    def forward(
        self,
        input_ids: torch.LongTensor,
        pixel_values: torch.FloatTensor,
        attention_mask: Optional[torch.LongTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        labels: Optional[torch.LongTensor] = None
    ):
        # Encode text
        text_output = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True,
        )
        if "pooler_output" in text_output:          # not all pre-trained models have pooling, BERT has
            text_feat = text_output.pooler_output   # model to handle different pre-trained encoders by checking
        else:
            text_feat = text_output.last_hidden_state[:, 0]  # CLS token fallback

        # Encode image
        image_output = self.image_encoder(
            pixel_values=pixel_values,
            return_dict=True,
        )
        if "pooler_output" in image_output:
            image_feat = image_output.pooler_output
        else:
            image_feat = image_output.last_hidden_state[:, 0]  # CLS token fallback

        # Fuse + classify
        fused = self.fusion(torch.cat([text_feat, image_feat], dim=1))
        logits = self.classifier(fused)

        output = {"logits": logits}
        if labels is not None:
            output["loss"] = self.criterion(logits, labels)

        return output # Returns logit values associated with each label


In [5]:
import torch

# Assign device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load config
config = torch.load("/content/drive/MyDrive/FinalProject/myenv/Demo-VQA_Bert-DEiT_config.pt")

# Reconstruct model
model = MultimodalVQAModel(**config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at facebook/deit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
'''
from transformers import ViltForQuestionAnswering, ViltProcessor

model = ViltForQuestionAnswering.from_pretrained("/content/drive/MyDrive/FinalProject/vilt_checkpoint")
processor = ViltProcessor.from_pretrained("/content/drive/MyDrive/FinalProject/vilt_checkpoint")
'''


In [6]:
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split

# Define base directory pointing to your Drive location
base_dir = "/content/drive/MyDrive/FinalProject/dataset"
os.makedirs(base_dir, exist_ok=True)

# Define regex to extract image ID
image_pattern = re.compile(r"( (in |on |of )?(the |this )?(image\d*) \?)")

# Read the raw Q&A file
# Extracts image ID and question at index[i] together with answer at [i+1]
# and organizes it into a pandas DataFrame, creates a list of unique answers,
# and then splits the dataset into training and testing sets, saving them as
# CSV.

qa_file = os.path.join(base_dir, "all_qa_pairs.txt")
with open(qa_file, "r", encoding="utf-8") as f: # open in read mode with in utf-8 encoding
    qa_data = [x.strip() for x in f.readlines()] # x.strip() removes white spaces

records = []
for i in range(0, len(qa_data), 2): # iterates in steps of two because quesiton is at index i and answer at i+1
    match = image_pattern.findall(qa_data[i]) # extract image ID from question string
    if match:
        img_id = match[0][3]
        question = qa_data[i].replace(match[0][0], "").strip() # remove image ID and replace question
        answer = qa_data[i + 1].strip() # answer retrieved from next element
        records.append({"question": question, "answer": answer, "image_id": img_id}) #populate dictionary



df = pd.DataFrame(records)


answer_space = []
for ans in df["answer"].to_list():
    if "," in ans:
        answer_space += ans.replace(" ", "").split(",")
    else:
        answer_space.append(ans)

# sort and remove duplicates from answer_space
answer_space = sorted(set(answer_space))

# Write the answer space to file
with open(os.path.join(base_dir, "answer_space.txt"), "w", encoding="utf-8") as f:
    f.writelines("\n".join(answer_space))

# Split dataset into training and evaluation
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save to CSV
train_df.to_csv(os.path.join(base_dir, "data_train.csv"), index=False)
test_df.to_csv(os.path.join(base_dir, "data_eval.csv"), index=False)


In [7]:
from copy import deepcopy
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from datasets import load_dataset
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import re
import pandas as pd
import os
from transformers import (
    # Preprocessing / Common
    AutoTokenizer, AutoFeatureExtractor,
    # Text & Image Models & transformers (ViTModel, DeiTModel, BEiT)
    AutoModel,
    # Training / Evaluation
    TrainingArguments, Trainer,
    # Misc
    logging
)

# import nltk
# nltk.download('wordnet')

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

# SET CACHE FOR HUGGINGFACE TRANSFORMERS + DATASETS
os.environ['HF_HOME'] = os.path.join(".", "cache")
# SET ONLY 1 GPU DEVICE
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

#set_caching_enabled(True)>> this line deleted as 'datasets' no longer supports explicit cahce enabling
#set_caching_enabled(True)
logging.set_verbosity_error()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
from nltk.corpus import wordnet as wn

def wup_measure(pred, gt, threshold=0.9):
    pred_synsets = wn.synsets(pred)
    gt_synsets = wn.synsets(gt)

    if not pred_synsets or not gt_synsets:
        return 0.0

    max_score = max((s1.wup_similarity(s2) or 0) for s1 in pred_synsets for s2 in gt_synsets)
    return 1.0 if max_score >= threshold else max_score

In [10]:

from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
})


In [12]:
from transformers import AutoTokenizer, AutoImageProcessor

# Rebuild processors from original names
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
image_processor = AutoImageProcessor.from_pretrained("facebook/deit-base-patch16-224")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [13]:
tokenizer.save_pretrained("/content/drive/MyDrive/FinalProject/myenv/tokenizer/")
image_processor.save_pretrained("/content/drive/MyDrive/FinalProject/myenv/image_processor/")


['/content/drive/MyDrive/FinalProject/myenv/image_processor/preprocessor_config.json']

In [14]:
from transformers import AutoTokenizer, AutoImageProcessor

# Load back
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/FinalProject/myenv/tokenizer/")
image_processor = AutoImageProcessor.from_pretrained("/content/drive/MyDrive/FinalProject/myenv/image_processor/")


In [34]:
def showExample(train=True, id=None):
    if train:
        data = dataset["train"]
    else:
        data = dataset["test"]

    if id is None:
        id = np.random.randint(len(data))

    img_path = os.path.join(image_dir, data[id]["image_id"] + ".png")

    if not os.path.exists(img_path):
        print(f"⚠️ Image not found at: {img_path}")
        return

    label = answer_space.index(data[id]["answer"]) if data[id]["answer"] in answer_space else "N/A"

    # Load question and image
    question = data[id]["question"]
    image = Image.open(img_path).convert("RGB")

    # 🔠 Tokenize question
    text_inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)

    # 🖼️ Process image
    image_inputs = image_processor(images=image, return_tensors="pt")

    # 🧩 Merge inputs
    inputs = {
        "input_ids": text_inputs["input_ids"],
        "attention_mask": text_inputs["attention_mask"],
        "pixel_values": image_inputs["pixel_values"]
    }

    # 🔮 Run model
    with torch.no_grad():
        outputs = model(**inputs)

    # 📈 Predict
    logits = outputs["logits"]  # fixed: dictionary-style access
    predicted_idx = logits.argmax(-1).item()
    predicted_answer = answer_space[predicted_idx]
    wup_score = wup_measure(pred=predicted_answer, gt=data[id]["answer"])

    # 🎨 Display result
    print(f"✅ Found image at: {os.path.abspath(img_path)}")
    display(image)
    print("Question:\t", question)
    label = data[id].get("label", "N/A")  # fallback if key is missing
    #print("Answer:\t\t", data[id]["answer"], f"(Label: {label})")
    print("Answer:\t\t", data[id]["answer"])
    print("Predicted answer:", predicted_answer)
    print("WUP Score:", wup_score)


In [ ]:
showExample()

In [16]:
from datasets import load_dataset
from IPython.display import display
from PIL import Image
import numpy as np
import os

# Set the correct image directory in Google Drive
image_dir = "/content/drive/MyDrive/FinalProject/dataset/images"

def showExample(train=True, id=None):
    if train:
        data = dataset["train"]
    else:
        data = dataset["test"]

    if id is None:
        id = np.random.randint(len(data))

    img_path = os.path.join(image_dir, data[id]["image_id"] + ".png")

    if not os.path.exists(img_path):
        print(f"⚠️ Image not found at: {img_path}")
        return


    # Define question and image path
    question = data[id]["question"]
    #  image_path = "/content/drive/MyDrive/FinalProject/dataset/images/image3.png"

    # Load image as RGB PIL object
    image = Image.open(img_path).convert("RGB")

    # Preprocess inputs
    inputs = image_processor(text=question, images=image, return_tensors="pt")

    # Run model
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predicted answer index and label
    logits = outputs.logits
    predicted_idx = logits.argmax(-1).item()
    predicted_answer = answer_space[predicted_idx]
    wup_score = wup_measure(pred=predicted_answer, gt=data[id]["answer"])

    print(f"✅ Found image at: {os.path.abspath(img_path)}")
    #  image = Image.open(img_path)
    display(image)

    print("Question:\t", question)
    print("Answer:\t\t", data[id]["answer"], f"(Label: {data[id]['label']})")
    print("Predicted answer:", predicted_answer)
    print("WUP Score:", wup_score)

# Show a random training example
showExample()

/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'text'
  return self.preprocess(images, **kwargs)


TypeError: MultimodalVQAModel.forward() missing 1 required positional argument: 'input_ids'

In [31]:
from PIL import Image
import torch
from transformers import ViltProcessor, ViltForQuestionAnswering

# Load your trained model and processor
# model = ViltForQuestionAnswering.from_pretrained("path_to_your_saved_model")
# processor = ViltProcessor.from_pretrained("path_to_your_saved_model")

# Load answer space from file
answer_space_path = "/content/drive/MyDrive/FinalProject/dataset/answer_space.txt"
with open(answer_space_path, "r", encoding="utf-8") as f:
    answer_space = f.read().splitlines()

# Define question and image path
question = "What is the color of the chair in front of the white wall?"
image_path = "/content/drive/MyDrive/FinalProject/dataset/images/image3.png"

# Load image as RGB PIL object
image = Image.open(image_path).convert("RGB")

# Preprocess inputs
inputs = image_processor(text=question, images=image, return_tensors="pt")

# Run model
with torch.no_grad():
    outputs = model(**inputs)

# Get predicted answer index and label
logits = outputs.logits
predicted_idx = logits.argmax(-1).item()
predicted_answer = answer_space[predicted_idx]

print("Predicted answer:", predicted_answer)


/usr/local/lib/python3.11/dist-packages/transformers/image_processing_utils.py:42: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'text'
  return self.preprocess(images, **kwargs)


TypeError: MultimodalVQAModel.forward() missing 1 required positional argument: 'input_ids'